In [ ]:
import tensorflow as tf
import numpy as np
import random
import time
import argparse
from math import sqrt

try:
    from config import config
    from DilatedRNN import DilatedRNNWavefunction
    from utils import Fullyconnected_localenergies, Fullyconnected_diagonal_matrixelements
    from vca import vca_solver
    print("Successfully imported local modules.")
except ImportError:
    print("Local .py files not found. Please ensure they are in the same directory or paste the classes below.")

tf.compat.v1.disable_eager_execution()

# Section 1: Ising Model as Energy Minimization

## 1.1 What is the Ising Model?
The Ising model was originally developed in statistical physics to describe the **ferromagnetism** of solid materials. It assumes that a system consists of a collection of **spins** arranged on a lattice, where each spin can only take one of two discrete values:
* $s_i = +1$ (Spin Up)
* $s_i = -1$ (Spin Down)

In this implementation, the spins are mapped to binary values $s \in \{0, 1\}$ to be compatible with neural network processing.

## 1.2 Mathematical Nature & The Hamiltonian
The **Hamiltonian** is a function in physics that represents the total energy of a system. For a fully connected Ising model, the energy function $E(\mathbf{s})$ is defined as:

$$E(\mathbf{s}) = -\sum_{i < j} J_{ij} \sigma_i \sigma_j - \sum_i h_i \sigma_i$$

* **$J_{ij}$ (Interaction Term)**: Defines the coupling strength between spins $i$ and $j$.
    * If $J_{ij} > 0$: Spins tend to align in the same direction (Ferromagnetic).
    * If $J_{ij} < 0$: Spins tend to align in opposite directions (Anti-ferromagnetic).
* **$h_i$ (External Field)**: Represents the influence of an external magnetic field or bias on an individual spin. In this code, a transverse field `Bx` is introduced to add non-diagonal perturbations.
* **Code Example**: In `utils.py`, the `Jz` matrix stores all interaction weights, and the function `Fullyconnected_diagonal_matrixelements` calculates the classical energy based on these weights.



In [ ]:
def Fullyconnected_diagonal_matrixelements(Jz, samples):
    numsamples = samples.shape[0]
    N = samples.shape[1]
    energies = np.zeros((numsamples), dtype = np.float64)

    for i in range(N-1):
      values = np.expand_dims(samples[:,i], axis = -1)+samples[:,i+1:]
      valuesT = np.copy(values)
      valuesT[values==2] = +1 #If both spins are up
      valuesT[values==0] = +1 #If both spins are down
      valuesT[values==1] = -1 #If they are opposite

      energies += np.sum(valuesT*(-Jz[i,i+1:]), axis = 1)

    return energies

## 1.3 Problem Type: Combinatorial Optimization
From a computer science perspective, the Ising model is a classic **Combinatorial Optimization Problem**:
* **Goal**: To find a specific arrangement among $2^N$ possible discrete configurations that minimizes the total energy of the system.
* **Applications**: Many famous NP-hard problems, such as the **Max-Cut Problem**, **Traveling Salesperson Problem (TSP)**, or **Graph Coloring**, can be exactly mapped onto the energy minimization of an Ising model.

## 1.4 The Essence of NP-hardness
Finding the lowest energy state (the **Ground State**) of an Ising model is extremely difficult for the following reasons:

1.  **Exponential State Space**: For $N$ spins, there are $2^N$ possible states. Brute-force search becomes impossible as $N$ grows.
2.  **Frustration**: When the distribution of $J_{ij}$ is complex (as in Spin Glass models), the system cannot satisfy all coupling terms simultaneously leading towards numerous equivalent ground states.
3.  **Complex Energy Landscape**: Frustration leads to a highly non-convex energy landscape filled with numerous **Local Minima**.
4.  **Mathematical Bottleneck**: Due to these high energy barriers, traditional optimization algorithms easily get stuck in local optima, failing to find the global optimum. This is the core of its NP-hardness.

# Section 2:Simulated Annealing as a Classical Heuristic

## 2.1 What is Simulated Annealing
Simulated Annealing (SA) is a classical **probabilistic optimization algorithm** designed to find low-energy (or high-quality) solutions in large, discrete, and non-convex search spaces. It was originally inspired by the physical annealing process in metallurgy, but in optimization it should be understood primarily as a **randomized search heuristic** rather than a physical simulation.

Formally, simulated annealing defines a **discrete-time, inhomogeneous Markov chain** over the solution space. Starting from an initial solution, the algorithm repeatedly proposes local moves within a predefined neighborhood structure. If a proposed move improves the objective value, it is always accepted. Otherwise, the move may still be accepted with a probability that depends on both the **increase in cost** and a **temperature parameter** that decreases over time.

The key feature of simulated annealing is its ability to **accept worse solutions during the early stages of the search**, which allows the algorithm to escape local minima that commonly trap greedy or purely descent-based methods. As the temperature is gradually lowered according to a cooling schedule, the algorithm becomes increasingly conservative and focuses on refining high-quality solutions.

From a theoretical perspective, the acceptance rule of simulated annealing is closely related to the **Boltzmann (Gibbs) distribution**. When the temperature is held fixed, the induced Markov chain admits a Boltzmann distribution as its invariant distribution. As the temperature approaches zero, this distribution concentrates on the set of global minima of the objective function. A rigorous analysis of this viewpoint and the convergence properties of simulated annealing can be found in the classical work of Bertsimas and Tsitsiklis (1993).

## 2.2 How Simulated Annealing works

This section reconstructs the simulated annealing procedure by following the basic elements defined in the classical formulation of Bertsimas and Tsitsiklis (1993).

### Solution Space and Objective

Simulated annealing operates on a **finite set of feasible solutions**, denoted by $S$.  
Each solution represents a valid configuration of the problem (e.g., a spin configuration in the Ising model).

An objective (or energy) function $J$ is defined on $S$, assigning a real-valued cost to each solution.  
The goal of the algorithm is to find solutions with **minimal objective value**, ideally belonging to the set of global minima.

### Neighborhood Structure

For each solution $i \in S$, a **neighborhood set** $S(i)$ is defined.  
This set specifies which solutions can be reached from $i$ through a single local modification.

The neighborhood structure determines how the algorithm explores the solution space and encodes the notion of a *local move*.

### Randomized Move Proposal

At each iteration, the algorithm selects a candidate solution from the neighborhood of the current solution.  
This selection is randomized according to a fixed probability rule, ensuring that the search does not follow a deterministic path.

This randomness is essential for exploration and allows the algorithm to visit different regions of the solution space.

### Temperature and Acceptance Behavior

The algorithm maintains a **temperature parameter** that changes over time according to a predefined cooling schedule.  
The temperature controls how willing the algorithm is to accept worse solutions.

- At high temperature, the algorithm is more exploratory and may accept worse solutions frequently.
- At low temperature, the algorithm becomes more conservative and favors improvements.

This mechanism enables simulated annealing to escape local minima early in the search and gradually refine solutions as the temperature decreases.

### State Evolution

Starting from an initial solution, the algorithm repeatedly:
1. Proposes a neighboring solution at random,
2. Evaluates its objective value,
3. Decides whether to accept or reject the move based on the current temperature.

Through this process, the algorithm generates a sequence of solutions that gradually concentrates on low-energy regions of the solution space.

---

**Remark.**  
From a theoretical perspective, this iterative process defines a time-inhomogeneous Markov chain over the solution space. When the temperature is held constant, the long-run behavior of the algorithm favors low-cost solutions, providing the foundation for simulated annealing as a global optimization heuristic.

## 2.2.1 Boltzmann Distribution (Intuition)

This section provides additional intuition for the **acceptance behavior** described in the previous section.  
Rather than introducing a new algorithmic step, it explains *why* simulated annealing uses a temperature-dependent probabilistic acceptance rule.

### Preference for Low-Energy Solutions

The Boltzmann distribution expresses a simple and intuitive idea:

**solutions with lower energy (or lower cost) are more likely than solutions with higher energy**.

This preference is controlled by a temperature parameter:
- At **high temperature**, differences in energy matter less, and many solutions are treated similarly.
- At **low temperature**, low-energy solutions are strongly preferred over higher-energy ones.

This temperature-dependent preference mirrors how physical systems behave under thermal fluctuations.

### Connection to Simulated Annealing

In simulated annealing, the Boltzmann distribution is not used for explicit sampling.  
Instead, it provides a **conceptual explanation** for the acceptance rule used during the search.

- Better solutions are always accepted.
- Worse solutions may still be accepted, especially when the temperature is high.

This behavior allows the algorithm to move uphill in the objective landscape early in the search, helping it escape local minima.

As the temperature decreases, the probability of accepting worse solutions diminishes, and the algorithm gradually focuses on improving the current solution.

### Boltzmann Form (Interpretation)

When the temperature is held fixed, the search process favors solutions according to a Boltzmann-like preference:

$$
\pi(i) \propto \exp\!\left(-\frac{J(i)}{T}\right),
$$

where $J(i)$ denotes the objective (or energy) of solution $i$, and $T$ is the temperature.

This distribution is **not computed explicitly** during the algorithm.  
Rather, the local acceptance rule of simulated annealing is designed so that the overall search behavior is consistent with this preference.

### Key Takeaway

**The Boltzmann distribution explains why simulated annealing behaves the way it does.**  
It provides a theoretical lens for understanding how temperature controls exploration and exploitation, while the algorithm itself remains a simple, local, randomized search procedure.


# Section 3: From Simulated Annealing to Learing

## 3.1 Limitations of Simulated Annealing
Simulated annealing is a well-established classical heuristic, but it exhibits several limitations when applied to large-scale and highly structured optimization problems.

The search behavior of simulated annealing is **manually designed**.
Key components such as neighborhood moves, acceptance rules, and temperature schedules are specified a priori and remain fixed, limiting the algorithm’s ability to adapt to instance-specific structure.

Moreover, simulated annealing does not **accumulate or reuse experience**.
Each run is independent, and information obtained during previous searches is not leveraged to improve future performance.

In rugged energy landscapes such as spin glass systems, the resulting random walk may spend a substantial amount of time exploring suboptimal regions before reaching low-energy configurations.

These limitations motivate learning-based approaches, which aim not to eliminate randomness, but to **learn heuristics that more effectively guide the search process**.

## 3.2 Why learning methods make sense


Compared to classical simulated annealing, learning-based methods offer several practical advantages when applied to large-scale NP-hard optimization problems. Simulated annealing relies on fixed, hand-designed components such as neighborhood moves and temperature schedules, which remain unchanged across different problem instances. In contrast, learning-based methods can adapt the search strategy by extracting patterns from data or repeated search experience, allowing the heuristic to better align with the underlying structure of the problem. 

Moreover, while simulated annealing treats each run independently, learning-based approaches can amortize computational cost across multiple instances by reusing learned knowledge, enabling faster generation of high-quality candidate solutions at inference time. This shift from instance-specific random exploration to experience-driven guidance is particularly beneficial in complex energy landscapes, where naive random walks may suffer from slow exploration and high computational cost. Importantly, learning-based methods do not remove randomness or circumvent the NP-hard nature of the problem, but they can significantly improve practical efficiency by guiding randomized search toward more promising regions of the solution space.

# Section 4: VNA as Example

Here is the link to VNA Github: https://github.com/VectorInstitute/VariationalNeuralAnnealing/tree/main?tab=readme-ov-file

## 4.1 Motivation

Simulated annealing solves optimization problems by approximately sampling a sequence of Boltzmann distributions, but its effectiveness is often limited by slow sampling dynamics in rugged or glassy energy landscapes. In practical, finite-time runs, the system deviates from equilibrium, leading to suboptimal solutions.

Variational Neural Annealing addresses this limitation by replacing the exact Boltzmann distribution with a flexible, parameterized model that can be efficiently sampled. Instead of annealing the physical system itself, VNA anneals a learnable probability distribution whose parameters are optimized using variational principles and energy feedback. Modern autoregressive neural networks provide an ideal parameterization, as they allow exact and fast sampling even when representing complex, highly structured landscapes. This shift from trajectory-based sampling to distribution-based learning enables more efficient exploration of low-energy configurations under limited computational budgets.

## 4.2 Algorithm Logic

This section provides a step-by-step walkthrough of a learning-based annealing method for Ising optimization.  
Each conceptual step is explicitly linked to a corresponding function in the implementation, allowing readers to connect algorithmic ideas with concrete code components.



### 1. Load an Ising Instance

**Relevant functions:**  
- `config.read_graph(graph_path)`  
- `config.__init__()`

We begin by loading an Ising problem instance from file. The graph structure and coupling strengths are encoded in a dense interaction matrix , which defines the Ising Hamiltonian. This step specifies the optimization problem and is independent of the solver.

In [ ]:
# Initialize the configuration with a specific Ising instance
instance_path = "dataset/EA/EA_10x10/10x10_uniform_seed1.txt" 
cfg = config(instance_path, seed=42)

# Verify the system scale and problem dimensions
print(f"System Size (N): {cfg.N}") # Number of spins (sites)
print(f"J-Matrix Shape: {cfg.Jz.shape}") # Should be an (N x N) matrix

# Physical Consistency Check: The interaction J_ij must equal J_ji (Symmetry)
# In physics, the coupling between spin i and spin j is mutual.
is_symmetric = (cfg.Jz == cfg.Jz.T).all()
print(f"Is the J-matrix symmetric? {is_symmetric}")

# Inspect a sub-section of the interaction matrix
# This represents a local 'snapshot' of the couplings between the first 5 spins.
print("\nLocal Sampling of J-Matrix (Top 5x5):")
print(cfg.Jz[:5, :5])

### 2. Define the Neural Sampler

**Relevant class:**  
- `DilatedRNNWavefunction`

The neural sampler parameterizes a probability distribution over spin configurations. We use an autoregressive recurrent neural network to model the joint distribution of spins, enabling exact and efficient sampling without Markov chain dynamics. The network serves as a learnable heuristic that guides the randomized search process.

In [ ]:
# Each RNN layer uses the same number of hidden units.
# cfg.num_layers controls how many dilated layers are used.
# cfg.num_units controls the capacity of each RNN cell.
units = [cfg.num_units] * cfg.num_layers
# Instantiate the autoregressive neural sampler
rnn_sampler = DilatedRNNWavefunction(
    systemsize=cfg.N,                 # Number of spins in the Ising model
    units=units,                      # Hidden units per RNN layer
    layers=cfg.num_layers,            # Number of dilated RNN layers
    activation=cfg.activation_function,
    seed=cfg.seed
)

print("Number of spins (N):", cfg.N)
print("Number of RNN layers:", cfg.num_layers)
print("Hidden units per layer:", cfg.num_units)
print("RNN cells per layer:", len(rnn_sampler.rnn[0]))
print("Total number of RNN cells:", cfg.num_layers * cfg.N)
#Each layer contains one RNN cell per spin position.  
#This design allows the model to capture both short-range and long-range dependencies between spins through dilated connections.

### 3. Sampling Spin Configurations

**Relevant method:**  
- `DilatedRNNWavefunction.sample(numsamples, inputdim)`

At each iteration, the model generates a batch of complete spin configurations by sampling from the learned distribution. Each sample corresponds to a candidate solution to the Ising optimization problem. Randomness is preserved, but the sampling is biased by the learned model parameters.

---

In [ ]:
# Number of configurations to sample at each iteration
num_samples = cfg.numsamples

# IMPORTANT:
# The following call builds the sampling operations in the
# TensorFlow 1.x computation graph.
# `samples` and `log_probs` are symbolic Tensors, not concrete values yet.
samples, log_probs = rnn_sampler.sample(
    numsamples=num_samples,
    inputdim=2
)

print("Symbolic sample tensor shape:", samples.shape)
print("Symbolic log-probability tensor shape:", log_probs.shape)

# In TensorFlow 1.x, tensors must be explicitly evaluated
# inside a Session to obtain NumPy arrays.
with tf.compat.v1.Session(graph=rnn_sampler.graph) as sess:
    sess.run(tf.compat.v1.global_variables_initializer())
    
    # Run the sampling operation
    samples_val, log_probs_val = sess.run([samples, log_probs])

# At this point, `samples_val` and `log_probs_val` are NumPy arrays
print("Sample shape (NumPy):", samples_val.shape)
print("First 5 sampled configurations:")
print(samples_val[:5])



### 4. Energy Evaluation

**Relevant functions:**  
- `Fullyconnected_diagonal_matrixelements(Jz, samples)`  
- `Fullyconnected_localenergies(...)` (optional)

The quality of sampled configurations is evaluated using the Ising Hamiltonian. Energy values provide the only feedback signal used to guide learning. Lower-energy configurations correspond to better approximate solutions.

---

In [ ]:
# Jz encodes the pairwise couplings of the Ising Hamiltonian
Jz = cfg.Jz

# Compute classical Ising energies for each sampled configuration
energies = Fullyconnected_diagonal_matrixelements(Jz, samples_val)

print("Energy array shape:", energies.shape)
print("Mean energy:", energies.mean())
print("Minimum energy:", energies.min())

### 5. Training with Annealing

**Relevant function:**  
- `vca_solver(config)`

Model parameters are updated using energy feedback under a temperature-controlled objective. A warmup phase stabilizes learning, followed by an annealing schedule that gradually shifts the focus from exploration to exploitation. Unlike classical simulated annealing, annealing here regulates the learning objective rather than the sampling dynamics.

Training begins with a **warmup phase**, during which the model stabilizes before annealing is applied.  
This is followed by an **annealing phase**, where the temperature is gradually reduced. At higher temperatures, the learning objective encourages exploration across diverse configurations. As the temperature decreases, updates increasingly focus on concentrating probability mass around low-energy regions.

A key distinction from classical simulated annealing is that annealing in VCA does not control the sampling dynamics. Instead, it regulates the learning objective, acting as a curriculum that guides how the sampling distribution evolves over time.

### 6. Final Sampling for Benchmarking

**Relevant code block:**  
- Final sampling section in `vca_solver`

After the annealing process completes, the trained model is used to generate a large number of spin configurations efficiently. The minimum and average energies of these samples are reported as benchmark metrics, reflecting the practical performance of the learned heuristic.

## 4.3 Execution Cell
To run the whole solver in this Notebook, use the following code block. Ensure you have the problem instance file (e.g., `10x10_uniform_seed1.txt`) in your working directory.

In [ ]:
# 1. Path to your problem instance
instance_path = "dataset/EA/EA_10x10/10x10_uniform_seed1.txt" 
seed = 0

# 2. Initialize configuration
vca_config = config(instance_path, seed)

# 3. Run the solver
# This will output the annealing progress, energy (E), and free energy (F)
mean_energies, min_energies = vca_solver(vca_config)

print(f"\nFinal Results:")
print(f"Minimum Energy Found: {min_energies}")
print(f"Mean Energy: {mean_energies}")

**Reference**

D. Bertsimas and J. N. Tsitsiklis, *Simulated Annealing*, Statistical Science, vol. 8, no. 1, pp. 10–15, 1993.  
Available at: https://www.mit.edu/~dbertsim/papers/Optimization/Simulated%20annealing.pdf